In [1]:
import pandas as pd
import numpy as np

In [2]:
cali_layoffs = pd.read_csv('california_warn_raw.csv')
cali_population = pd.read_csv('county_population.csv', delimiter='\t', thousands=',')

population data from: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage_242301767

In [3]:
cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary


In [4]:
cali_layoffs.tail()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
9502,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No County Name,86.0,Layoff Permanent
9503,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No County Name,49.0,Closure Permanent
9504,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No County Name,11.0,Closure Permanent
9505,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No County Name,18.0,Layoff Unknown at this time
9506,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No County Name,50.0,Closure Permanent


## California 2020 Layoffs

We are selecting 2020 records by isolating the year on the `effective date` column.
By choosing `effective date` we make sure that these layoffs actually occurred in 2020, as some layoff notices can be issues much earlier the layoff date.

In [5]:
cali_layoffs['Year'] = cali_layoffs['Effective Date'].str[-4:]
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.replace('at thistime', 'at this time')
cali_2020 = cali_layoffs[cali_layoffs['Year'] == '2020']
cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')
cali_2020.tail()

<ipython-input-5-957d264fb98a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')


,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
5725,10/30/2019,01/03/2020,11/01/2019,"Ashford University, LLC",San Diego,San Diego County,218,Layoff Permanent,2020
5727,11/01/2019,01/10/2020,11/01/2019,American General Life Insurance Company,Pasadena,Los Angeles County,40,Closure Permanent,2020
5744,10/24/2019,01/01/2020,10/28/2019,"Hotel Del Coronado, Curio Collection By Hilton...",Coronado,San Diego County,160,Layoff Temporary,2020
5747,10/17/2019,05/24/2020,10/23/2019,Wells Fargo,Concord,Contra Costa County,357,Layoff Permanent,2020
5789,09/26/2019,01/16/2020,09/30/2019,"Forever 21 Logistics, LLC",Los Angeles,Los Angeles County,1170,Closure Permanent,2020


In [6]:
cali_2020['County'] = cali_2020['County'].str.strip()
cali_2020.loc[~(cali_2020['County'].str.endswith('County')),'County'] = cali_2020['County'] + " County"

<ipython-input-6-0b43f9e4a6a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2020['County'] = cali_2020['County'].str.strip()


In [7]:
cali_population.head(5)

,California,"39,512,223"
0,".Alameda County, California",1671329
1,".Alpine County, California",1129
2,".Amador County, California",39752
3,".Butte County, California",219186
4,".Calaveras County, California",45905


In [8]:
cali_population.rename(columns={'California':'Counties','39,512,223':'Population'}, inplace=True)
cali_population['County Names'] = cali_population['Counties'].str[1:]
cali_population['County Names'] = cali_population['County Names'].str.split(',').str[0]
cali_population.head(4)

,Counties,Population,County Names
0,".Alameda County, California",1671329,Alameda County
1,".Alpine County, California",1129,Alpine County
2,".Amador County, California",39752,Amador County
3,".Butte County, California",219186,Butte County


In [9]:
len(cali_2020)

5692

In [10]:
cali_merge = pd.merge(cali_2020, cali_population, how="left", left_on="County", right_on="County Names")
cali_merge.drop(columns='Counties', inplace=True)
cali_merge.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year,Population,County Names
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79,Closure Permanent,2020,10039107,Los Angeles County
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608,Closure Permanent,2020,10039107,Los Angeles County
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104,Closure Permanent,2020,3175692,Orange County
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66,Layoff Temporary,2020,2180085,San Bernardino County
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91,Layoff Temporary,2020,3338330,San Diego County


In [11]:
layoffs_by_county = cali_merge.groupby(['County Names','Year']).agg('sum')
layoffs = layoffs_by_county.merge(cali_population, how="left", on="County Names")
layoffs.drop(columns=['Counties', 'Population_x'], inplace=True)
layoffs.rename(columns={"Population_y":"Population"}, inplace=True)
layoffs['Percentage Laid Off'] = (layoffs['Employees']/layoffs['Population']) * 100
layoffs.sort_values(by='Percentage Laid Off', ascending=False)

,County Names,Employees,Population,Percentage Laid Off
18,Mariposa County,2262,17203,13.148869
33,San Francisco County,56820,881549,6.445473
21,Mono County,879,14444,6.085572
12,Inyo County,539,18039,2.987971
32,San Diego County,75794,3338330,2.270417
36,San Mateo County,16987,766573,2.215966
23,Napa County,2975,137744,2.159804
25,Orange County,65189,3175692,2.052749
0,Alameda County,32756,1671329,1.959877
15,Los Angeles County,193946,10039107,1.931905


In [12]:
cali_2019 = cali_layoffs[cali_layoffs['Year'] == '2019']
cali_2019['Employees'] = cali_2019['Employees'].astype('Int64')
cali_2019['County'] = cali_2019['County'].str.strip()
cali_2019.loc[~(cali_2019['County'].str.endswith('County')),'County'] = cali_2019['County'] + " County"
cali_2019

<ipython-input-12-d2d3be101e82>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2019['Employees'] = cali_2019['Employees'].astype('Int64')
<ipython-input-12-d2d3be101e82>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2019['County'] = cali_2019['County'].str.strip()
/Users/dilcia_mercedes/.local/share/virtualenvs/cali_WARN-MY_IDcvv/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
5514,04/29/2019,07/01/2019,03/03/2020,Kimberly-Clark Corporation,Fullerton,Orange County,204,Closure Permanent,2019
5528,05/02/2019,07/05/2019,02/25/2020,Alticor Inc.,Buena Park,Orange County,45,Layoff Permanent,2019
5643,12/20/2019,12/20/2019,12/30/2019,"Event Solutions International, Inc. dba MotusO...",Long Beach,Los Angeles County,38,Layoff Permanent,2019
5644,12/30/2019,12/31/2019,12/30/2019,Taylor Lane Organic Coffee,Petaluma,Sonoma County,6,Closure Permanent,2019
5645,10/16/2019,11/01/2019,12/27/2019,Barneys New York,San Francisco,San Francisco County,100,Closure Permanent,2019
...,...,...,...,...,...,...,...,...,...
6741,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",Ontario,San Bernardino County,16,Closure Permanent,2019
6742,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",San Diego,San Diego County,84,Closure Permanent,2019
6743,06/28/2019,08/30/2019,06/28/2019,"Coloredge, Inc.",Burbank,Los Angeles County,32,Closure Permanent,2019
6744,06/28/2019,08/31/2019,06/28/2019,Verity Business Services,San Jose,Santa Clara County,5,Layoff Permanent,2019


In [13]:
cali_merge_2019 = pd.merge(cali_2019, cali_population, how="left", left_on="County", right_on="County Names")
cali_merge_2019.drop(columns='Counties', inplace=True)
cali_merge_2019.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year,Population,County Names
0,04/29/2019,07/01/2019,03/03/2020,Kimberly-Clark Corporation,Fullerton,Orange County,204,Closure Permanent,2019,3175692,Orange County
1,05/02/2019,07/05/2019,02/25/2020,Alticor Inc.,Buena Park,Orange County,45,Layoff Permanent,2019,3175692,Orange County
2,12/20/2019,12/20/2019,12/30/2019,"Event Solutions International, Inc. dba MotusO...",Long Beach,Los Angeles County,38,Layoff Permanent,2019,10039107,Los Angeles County
3,12/30/2019,12/31/2019,12/30/2019,Taylor Lane Organic Coffee,Petaluma,Sonoma County,6,Closure Permanent,2019,494336,Sonoma County
4,10/16/2019,11/01/2019,12/27/2019,Barneys New York,San Francisco,San Francisco County,100,Closure Permanent,2019,881549,San Francisco County


In [14]:
layoffs_by_county_2019 = cali_merge_2019.groupby(['County Names','Year']).agg('sum')
layoffs_2019 = layoffs_by_county_2019.merge(cali_population, how="left", on="County Names")
layoffs_2019.drop(columns=['Counties', 'Population_x'], inplace=True)
layoffs_2019.rename(columns={"Population_y":"Population"}, inplace=True)
layoffs_2019['Percentage Laid Off'] = (layoffs_2019['Employees']/layoffs_2019['Population']) * 100
layoffs_2019.sort_values(by='Percentage Laid Off', ascending=False)

,County Names,Employees,Population,Percentage Laid Off
1,Butte County,1877,219186,0.856350
12,Nevada County,521,99755,0.522280
5,Glenn County,143,28393,0.503645
24,Santa Clara County,8882,1927852,0.460720
19,San Francisco County,3808,881549,0.431967
0,Alameda County,4816,1671329,0.288154
22,San Mateo County,1869,766573,0.243812
34,Yolo County,479,220500,0.217234
7,Kern County,1805,900202,0.200511
14,Placer County,769,398329,0.193056


In [15]:
layoff_closure_list = ['Layoff Permanent', 'Closure Permanent']
counties_list = ['Butte County', 'Santa Cruz County', 'Yolo County', 'Orange County', 'Los Angeles County', 'Humboldt County', 'Fresno County']

## Layoffs and Closures for Counties

In [16]:
high_rent_burden_counties_20 = cali_2020[cali_2020['County'].isin(counties_list)]
perm_layoffs_closures_20 = high_rent_burden_counties_20[high_rent_burden_counties_20['Layoff/Closure'].isin(layoff_closure_list)]
SIP_perm_layoffs_closures_20 = perm_layoffs_closures_20[perm_layoffs_closures_20['Notice Date'] > '03/19/2020']

high_rent_burden_counties_19 = cali_2019[cali_2019['County'].isin(counties_list)]
perm_layoffs_closures_19 = high_rent_burden_counties_19[high_rent_burden_counties_19['Layoff/Closure'].isin(layoff_closure_list)]
SIP_perm_layoffs_closures_19 = perm_layoffs_closures_19[perm_layoffs_closures_19['Notice Date'] > '03/19/2019']
SIP_perm_layoffs_closures_19.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
5514,04/29/2019,07/01/2019,03/03/2020,Kimberly-Clark Corporation,Fullerton,Orange County,204,Closure Permanent,2019
5528,05/02/2019,07/05/2019,02/25/2020,Alticor Inc.,Buena Park,Orange County,45,Layoff Permanent,2019
5643,12/20/2019,12/20/2019,12/30/2019,"Event Solutions International, Inc. dba MotusO...",Long Beach,Los Angeles County,38,Layoff Permanent,2019
5701,11/06/2019,11/10/2019,11/14/2019,"Park Holding, Inc.",Los Angeles,Los Angeles County,47,Closure Permanent,2019
5710,10/31/2019,12/31/2019,11/06/2019,Warner Music Inc.,Los Angeles,Los Angeles County,1,Closure Permanent,2019


In [17]:
layoff_closure_sums_2020 = SIP_perm_layoffs_closures_20.groupby(['County','Layoff/Closure']).sum()
layoff_closure_sums_2020.reset_index("Layoff/Closure")
layoff_closure_sums_2019 = SIP_perm_layoffs_closures_19.groupby(['County','Layoff/Closure']).sum()
layoff_closure_sums_2019.reset_index("Layoff/Closure")


,Layoff/Closure,Employees
County,,
Butte County,Closure Permanent,493
Butte County,Layoff Permanent,11
Fresno County,Closure Permanent,1160
Fresno County,Layoff Permanent,21
Los Angeles County,Closure Permanent,5398
Los Angeles County,Layoff Permanent,6194
Orange County,Closure Permanent,2353
Orange County,Layoff Permanent,1184
Santa Cruz County,Closure Permanent,312


In [18]:
sums = layoff_closure_sums_2020.merge(layoff_closure_sums_2019, left_on=["County", "Layoff/Closure"], right_on=["County", "Layoff/Closure"], how="left")
sums.rename(columns={'Employees_x':'Laid_Off_2020','Employees_y':'Laid_Off_2019'}, inplace=True)
sums

Laid_Off_2020  Laid_Off_2019
County             Layoff/Closure                                 
Butte County       Closure Permanent              8            493
                   Layoff Permanent             570             11
Fresno County      Closure Permanent            932           1160
                   Layoff Permanent             318             21
Los Angeles County Closure Permanent          10812           5398
                   Layoff Permanent           28080           6194
Orange County      Closure Permanent           1136           2353
                   Layoff Permanent           12091           1184
Santa Cruz County  Closure Permanent            113            312
                   Layoff Permanent             779           <NA>
Yolo County        Layoff Permanent             238             31

In [19]:
unemployed_sums_2020 = SIP_perm_layoffs_closures_20.groupby(['County']).sum()
unemployed_sums_2019 = SIP_perm_layoffs_closures_19.groupby(['County']).sum()

In [20]:
sums_2020_2019 = unemployed_sums_2020.merge(unemployed_sums_2019, how="left", left_on="County", right_on="County")
sums_2020_2019.rename(columns={'Employees_x':'Permanent L/C Total 2020','Employees_y':'Permanent L/C Total 2019'}, inplace=True)
sums_2020_2019

,Permanent L/C Total 2020,Permanent L/C Total 2019
County,,
Butte County,578,504
Fresno County,1250,1181
Los Angeles County,38892,11592
Orange County,13227,3537
Santa Cruz County,892,312
Yolo County,238,474


## Figures for Labor Expert Interview

In [21]:
sums

Laid_Off_2020  Laid_Off_2019
County             Layoff/Closure                                 
Butte County       Closure Permanent              8            493
                   Layoff Permanent             570             11
Fresno County      Closure Permanent            932           1160
                   Layoff Permanent             318             21
Los Angeles County Closure Permanent          10812           5398
                   Layoff Permanent           28080           6194
Orange County      Closure Permanent           1136           2353
                   Layoff Permanent           12091           1184
Santa Cruz County  Closure Permanent            113            312
                   Layoff Permanent             779           <NA>
Yolo County        Layoff Permanent             238             31

<!-- Hi! Here's what we have below:
- # of people laid off in 2020, and in 2019
- # of  -->

In [22]:
all_together = sums.merge(sums_2020_2019, how="left", left_on="County", right_on="County")
all_together = all_together.merge(cali_population, how='left', left_on='County', right_on='County Names')
all_together.drop(columns=['Counties'], inplace=True)
all_info = all_together[['County Names', 'Laid_Off_2020', 'Laid_Off_2019', 'Permanent L/C Total 2020', 'Permanent L/C Total 2019', 'Population']]
all_info

,County Names,Laid_Off_2020,Laid_Off_2019,Permanent L/C Total 2020,Permanent L/C Total 2019,Population
0,Butte County,8,493,578,504,219186
1,Butte County,570,11,578,504,219186
2,Fresno County,932,1160,1250,1181,999101
3,Fresno County,318,21,1250,1181,999101
4,Los Angeles County,10812,5398,38892,11592,10039107
5,Los Angeles County,28080,6194,38892,11592,10039107
6,Orange County,1136,2353,13227,3537,3175692
7,Orange County,12091,1184,13227,3537,3175692
8,Santa Cruz County,113,312,892,312,273213
9,Santa Cruz County,779,<NA>,892,312,273213


In [25]:
SIP_perm_layoffs_closures_19['Employees'].sum()

17600

In [27]:
SIP_perm_layoffs_closures_20['Employees'].sum()

55077